### differents $B_\theta$, petits $\theta$, differents $Bsf$

In [1]:
import os

In [2]:
%mkdir -p tmp

In [3]:
data_folder = os.path.join('..', 'Psychopy_data', 'pilot', 'B_theta_sf')#,'json')
print('data_folder =', data_folder)

data_folder = ../Psychopy_data/pilot/B_theta_sf


In [4]:
%mkdir -p {data_folder}

In [5]:
%%writefile tmp/exp.py
import sys
import numpy as np
import os
import imageio
import sys

print(sys.argv)
data_folder = sys.argv[1]
exp_observer = sys.argv[2]

exp_name = "Perception de l'orientation"
exp_info = {'participant': exp_observer,
            'exp_name': exp_name
           }

N_B_thetas = 5
N_B_Bsf = 5
B_thetas = np.pi*np.logspace(-5, -1.5, N_B_thetas, base=2)
B_sfs = np.logspace(-4, 0., N_B_Bsf, base=2)
theta_max = np.pi/6

seed = 42
np.random.seed(seed)
test_length = 500     # number of trials
fixation_length = 0.25  # length of fixation, in second
stim_length = 0.25   # length of stim, in second

fig_width = 21

fullscr = False
fullscr = True

#help(mc.envelope_gabor)

def generate_random_cloud(theta, B_theta, B_sf, downscale = 1):
    fname = 'tmp/%s_%s_%s.png' % (theta, B_theta, B_sf)
    if not os.path.isfile(fname):
        import MotionClouds as mc
        fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                                 B_V=0, theta=np.pi/2-theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite(fname, im[:, :, 0])
    return fname

from psychopy import visual, core, event, gui, data

#win = visual.Window([1400,800],fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])


mouse = event.Mouse(newPos=(0, 0), visible=False)
x0, y0 = mouse.getPos()
event.clearEvents()
instructions = """
At each trial you see one textured image, report the 
orientation you perceive.

Move the mouse to the right if it is clock-wise of the vertical,
move the mouse to the left if it is counter clock-wise of the vertical.

Move the mouse to begin the experiment.
"""
msg = visual.TextStim(win, text=instructions, alignHoriz='center', alignVert='center', color='black')
msg.draw()
win.flip()
while not mouse.mouseMoved(distance=.05): 
    event.clearEvents()
fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC = generate_random_cloud(np.pi/2, B_theta=np.pi/2, B_sf=0.1)
bitmap = visual.ImageStim(win, MC, mask='gauss', size=0.8, units='height', interpolate=True)
bitmap.autolog = False 

msg = visual.TextStim(win, text='?', color='black')

import time
start_time = time.time()
print("Test started")

ans_list = []
# generate all clouds
np.random.seed(seed)
for trial in range(test_length):
    theta = (2*np.random.rand()-1)*theta_max

    # MC generation
    i_B_theta = np.random.randint(0, N_B_thetas)
    B_theta = B_thetas[i_B_theta]
    i_B_sf = np.random.randint(0, N_B_Bsf)
    B_sf = B_sfs[i_B_sf]
    MC1 = generate_random_cloud(theta, B_theta=B_theta, B_sf=B_sf) 

# run experiment
np.random.seed(seed)
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()

    theta = (2*np.random.rand()-1)*theta_max

    # MC generation
    i_B_theta = np.random.randint(0, N_B_thetas)
    B_theta = B_thetas[i_B_theta]
    i_B_sf = np.random.randint(0, N_B_Bsf)
    B_sf = B_sfs[i_B_sf]
    MC = generate_random_cloud(theta, B_theta=B_theta, B_sf=B_sf) 
    bitmap.setImage(MC)

     # Times the trial
    while clock.getTime() < fixation_length + stim_length:
        if clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        elif clock.getTime() < fixation_length + stim_length: 
            bitmap.draw()
        win.flip()
        
    msg.draw()
    win.flip()

    mouse.setPos(newPos=(0, 0))
    x0, y0 = mouse.getPos()
    event.clearEvents()

    while not mouse.mouseMoved(distance=.025): 
        # this creates a never-ending loop
        # until we move the mouse
        event.clearEvents()

    x, y = mouse.getPos()
    if x < 0:
        answer = 'left'
    else: #elif x > 0:
        answer = 'right'
        if x == 0: print('this should not happen 😤')
    event.clearEvents() 

    ans = event.getKeys()       

    if ans == ['escape', 'q']:
        win.close()
        core.quit()

    correct = ((theta > 0) and answer == 'right') or ((theta < 0) and answer == 'left')
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ',  answer, '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi),'B_sf=', B_sf)   

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, answer, B_theta, B_sf, i_B_theta, i_B_sf])

print('saving')
import json
json.dump(ans_list, open( os.path.join(data_folder, 'Psychophys_discrim_%s.json' % exp_info['participant']),'w',encoding="utf8"))
event.clearEvents() 

print('exiting')
event.clearEvents() 
win.close()
core.quit()


Overwriting tmp/exp.py


In [6]:
%%writefile tmp/test.py
import sys
import numpy as np
import MotionClouds as mc
import os
import imageio
import sys

print(sys.argv)

from psychopy import visual, core, event, gui, data
fullscr = False
#win = visual.Window([1400,800],fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])

print('exiting')
event.clearEvents() 
win.close()
core.quit()


Overwriting tmp/test.py


In [7]:
#!ipython tmp/test.py {data_folder} LP